In [2]:
import numpy as np
from openfermion.chem import geometry_from_pubchem
from quri_parts.chem import cas
from quri_parts.pyscf.mol import get_spin_mo_integrals_from_mole
from quri_parts.openfermion.mol import get_qubit_mapped_hamiltonian
from quri_parts.openfermion.ansatz import TrotterUCCSD
from quri_parts.core.operator import get_sparse_matrix
from pyscf import gto, scf, mcscf, cc
from quri_parts_qsci import qsci
from quri_parts.chem.mol import ActiveSpaceMolecularOrbitals, ActiveSpace
from quri_parts.pyscf.mol import PySCFMolecularOrbitals
from quri_parts.core.state import quantum_state

from typing import Sequence

import numpy.typing as npt

# Generate Hamiltonian of AB (azobenzene)
1. Hamiltonian
2. Exact ground state energy
3. Approximated energy includes electron correlation effects from single and double excitations. 

In [3]:
def from_Hartree_to_eV(Hartree_energy):
    return Hartree_energy/27.21

In [6]:
from scipy.sparse.linalg import eigsh

atom = geometry_from_pubchem("azobenzene") 
mole = gto.M(atom=atom)
mf = scf.RHF(mole).run()

active_space = cas(4, 4)
hamiltonian, mapping = get_qubit_mapped_hamiltonian(
    *get_spin_mo_integrals_from_mole(mole, mf.mo_coeff, active_space)
)

vals, vecs = eigsh(get_sparse_matrix(hamiltonian).toarray(), k=4, which="SA")

casci_gs_e = np.min(vals)
ccsd = cc.CCSD(mf).run(verbose=0)

print("Hamiltonian: ", hamiltonian)
print(f"The exact ground state energy is: {from_Hartree_to_eV(casci_gs_e)} eV")
print(f"The coupled cluster singles and doubles energy is: {from_Hartree_to_eV(ccsd.e_tot)} eV")

converged SCF energy = -562.053344724957
Hamiltonian:  (-561.2163615469071+0j)*I + (0.07929207448087101+0j)*Z0 + (0.07929207448087101+0j)*Z1 + (0.041737792195591525+0j)*Z2 + (0.04173779219559153+0j)*Z3 + (-0.11119726823032403+0j)*Z4 + (-0.111197268230324+0j)*Z5 + (-0.08721715886289064+0j)*Z6 + (-0.08721715886289064+0j)*Z7 + (0.0548897971305265+0j)*Z0 Z1 + -0.0015964182704949286*X0 Z1 X2 + -0.0015964182704949286*Y0 Z1 Y2 + (0.029990484826448983+0j)*Z0 Z2 + (0.05354451192305687+0j)*Z0 Z3 + -1.9995608176371332e-05*X0 Z1 Z2 Z3 X4 + -1.9995608176371332e-05*Y0 Z1 Z2 Z3 Y4 + (0.049962972036554226+0j)*Z0 Z4 + (0.051003560119291025+0j)*Z0 Z5 + 0.014257630971931865*X0 Z1 Z2 Z3 Z4 Z5 X6 + 0.014257630971931865*Y0 Z1 Z2 Z3 Z4 Z5 Y6 + (0.042956653593804545+0j)*Z0 Z6 + (0.054897656325667496+0j)*Z0 Z7 + (0.05354451192305687+0j)*Z1 Z2 + -0.0015964182704949286*X1 Z2 X3 + -0.0015964182704949286*Y1 Z2 Y3 + (0.029990484826448983+0j)*Z1 Z3 + (0.051003560119291025+0j)*Z1 Z4 + -1.9995608176371332e-05*X1 Z2 Z3

In [123]:
from_Hartree_to_eV(vals-vals[0])

array([0.        , 0.00831725, 0.00837364, 0.00831725])

In [7]:
from quri_parts.chem.mol.models import OrbitalType

asmo = ActiveSpaceMolecularOrbitals(
    PySCFMolecularOrbitals(mole, mf.mo_coeff), active_space
)

forzen_orbs = [i for i in range(mole.nao) if asmo.orb_type(i) != OrbitalType.ACTIVE]

ccsd = cc.CCSD(mf, frozen=forzen_orbs).run(verbose=0)

In [9]:
from typing import Sequence

import numpy.typing as npt


def ccsd_param_to_circuit_param(
    uccsd: TrotterUCCSD,
    n_electrons: int,
    t1: npt.NDArray[np.complex128],
    t2: npt.NDArray[np.complex128],
) -> Sequence[float]:
    in_param_list = uccsd.param_mapping.in_params
    param_list = []

    for param in in_param_list:
        name_split = param.name.split("_")
        if name_split[0] == "s":
            _, i_str, j_str = name_split
            i, j = int(i_str), int(j_str) - n_electrons // 2
            param_list.append(t1[i, j])

        if name_split[0] == "d":
            _, i_str, j_str, a_str, b_str = name_split
            i, j, b, a = (
                int(i_str),
                int(j_str),
                int(b_str) - n_electrons // 2,
                int(a_str) - n_electrons // 2,
            )
            param_list.append(t2[i, j, a, b])
    return param_list


In [266]:
from quri_parts.qulacs.estimator import (
    create_qulacs_vector_concurrent_parametric_estimator,
)
from quri_parts.core.state import quantum_state, apply_circuit

TROTTER_STEPS = 1
USE_SINGLES = True
REDUCE_PARAMETER = True

uccsd = TrotterUCCSD(
        active_space.n_active_orb * 2,
        active_space.n_active_ele,
        trotter_number=TROTTER_STEPS,
        use_singles=USE_SINGLES,
        singlet_excitation=REDUCE_PARAMETER,
    )
param = ccsd_param_to_circuit_param(uccsd, active_space.n_active_ele, ccsd.t1, ccsd.t2)
hf_state = quantum_state(active_space.n_active_orb * 2, bits=2**active_space.n_active_ele - 1)
state = apply_circuit(uccsd, hf_state)
bound_state = state.bind_parameters(param)

In [12]:
estimator = create_qulacs_vector_concurrent_parametric_estimator()
estimate = estimator(hamiltonian, state, [param])

print("The coupled cluster singles and doubles energy is:", from_Hartree_to_eV(ccsd.e_tot))
print(
    "The Trotter unitary coupled cluster singles and doubles energy is:",
    from_Hartree_to_eV(estimate[0].value.real),
)
print("Error: ", from_Hartree_to_eV(ccsd.e_tot-estimate[0].value.real))

The coupled cluster singles and doubles energy is: -20.656290717085593
The Trotter unitary coupled cluster singles and doubles energy is: -20.6562907247294
Error:  7.64380410002866e-09


## Excited states

In [119]:
exc_bits = (2**active_space.n_active_ele - 1) ^ 0b0011000 
excited_state = quantum_state(active_space.n_active_orb * 2, bits=exc_bits) 
ex_state_circuit = apply_circuit(uccsd, excited_state)
bound1_state = ex_state_circuit.bind_parameters(param)


In [22]:
estimator = create_qulacs_vector_concurrent_parametric_estimator()
estimate = estimator(hamiltonian, ex_state_circuit, [param])
print(
    "Energy difference: ",
    from_Hartree_to_eV(ccsd.e_tot-estimate[0].value.real),
)

Energy difference:  -0.00851674620296459


# QSCI

In [323]:
from quri_vm import VM

from quri_parts.backend.devices import star_device
from quri_parts.backend.units import TimeUnit, TimeValue

from typing import Iterable

from quri_parts.circuit import ImmutableQuantumCircuit
from quri_parts.core.sampling import ConcurrentSampler, MeasurementCounts
from quri_parts.qulacs.sampler import create_qulacs_general_vector_sampler
from quri_parts_qsci import qsci

BASIS_STATES = None
TOTAL_SHOTS = 10000

sampler = create_qulacs_general_vector_sampler()

bound_state = state.bind_parameters(param)
eigs, _ = qsci(
    hamiltonian, [bound_state], sampler, total_shots=TOTAL_SHOTS, num_states_pick_out=BASIS_STATES
)


In [324]:
from typing import Sequence
import numpy as np
from scipy.optimize import minimize
from quri_parts.core.state import quantum_state

def create_state(bits, params):
    approx_circuit = uccsd.bind_parameters(params)
    approx_exc_state = quantum_state(approx_circuit.qubit_count, circuit=approx_circuit, bits=bits)
    return approx_exc_state

def get_cost(exc_bits: Sequence[int], sampler, hamiltonian, fixed_state=[]):
    def f(x):
        approx_exc_state = create_state(bits=exc_bits, params=x)
        quantum_states = fixed_state+[approx_exc_state]
        vals, _ = qsci(
            hamiltonian,
            quantum_states,
            sampler,
            total_shots=TOTAL_SHOTS,
            num_states_pick_out=BASIS_STATES
        )
        return np.sum(vals)
    

    return f

In [327]:
sampler = create_qulacs_general_vector_sampler()

exc_masks = [0b00000000, 0b00010001, 0b00100001, 0b01000001, 0b10000001, 0b00111100]
# exc_masks = [0b00000000, 0b00000000, 0b00000000, 0b00000000]
fixed_state = []
for exc_mask in exc_masks:
    bits = (2**active_space.n_active_ele - 1) ^ exc_mask
    cost = get_cost(
        exc_bits=bits,
        sampler=sampler,
        hamiltonian=hamiltonian,
        fixed_state=fixed_state
    )

    bounds = [(-10**-1, 10**-1) for _ in range(uccsd.parameter_count)]
    sol = minimize(
        cost,
        x0=np.random.uniform(-10**-2, 10**-2, uccsd.parameter_count),
        method="L-BFGS-B",
        bounds=bounds,
        options={"maxiter": 500000, "disp": True}
    )

    excited_state = quantum_state(active_space.n_active_orb * 2, bits=bits) 
    ex_state_circuit = apply_circuit(uccsd, excited_state)

    estimator = create_qulacs_vector_concurrent_parametric_estimator()
    estimate = estimator(hamiltonian, ex_state_circuit, [list(sol.x)])
    print(
        f"exc_mask: {exc_mask}",
        "Energy difference: ",
        from_Hartree_to_eV(estimate[0].value.real-ccsd.e_tot),
    )
    #fixed_state.append(create_state(bits, params=sol.x))


/home/string-3/anaconda3/envs/QC/lib/python3.10/site-packages/scipy/optimize/_numdiff.py:619: ComplexWarning: Casting complex values to real discards the imaginary part
  J_transposed[i] = df / dx


exc_mask: 0 Energy difference:  0.00017221890093470802
exc_mask: 17 Energy difference:  0.01180278155781947


/home/string-3/anaconda3/envs/QC/lib/python3.10/site-packages/scipy/optimize/_lbfgsb_py.py:433: ComplexWarning: Casting complex values to real discards the imaginary part
  _lbfgsb.setulb(m, x, low_bnd, upper_bnd, nbd, f, g, factr, pgtol, wa,


exc_mask: 33 Energy difference:  0.011652166075096173
exc_mask: 65 Energy difference:  0.013562504301333073
exc_mask: 129 Energy difference:  0.011802488611940849
exc_mask: 60 Energy difference:  0.025213276787249612


In [292]:
estimator = create_qulacs_vector_concurrent_parametric_estimator()
estimate = estimator(hamiltonian, ex_state_circuit, [list(sol.x)])
print(
    "Energy difference: ",
    from_Hartree_to_eV(estimate[0].value.real-ccsd.e_tot),
)

Energy difference:  0.0925024286129064


In [230]:
from_Hartree_to_eV(vals-ccsd.e_tot)

array([-2.96971287e-08,  8.31722436e-03,  8.37360885e-03,  8.31722436e-03])

In [210]:
bin(2**active_space.n_active_ele - 1)

'0b1111'

In [216]:
bin((2**active_space.n_active_ele - 1) ^ 0b101000)

'0b100111'

In [268]:
err = lambda x: abs(x / casci_gs_e - 1)
print(f"The exact GS energy is {from_Hartree_to_eV(casci_gs_e)}")
print(
    f"The coupled cluster energy is {from_Hartree_to_eV(ccsd.e_tot)}, with relative error {err(ccsd.e_tot)}"
)
print(
    f"The Trotter UCCSD energy is {from_Hartree_to_eV(estimate[0].value.real)}, with relative error {err(estimate[0].value.real)}"
)
print(f"The qsci gs energy is {from_Hartree_to_eV(eigs[0])}, with relative error {err(eigs[0])}")

The exact GS energy is -20.656290746782723
The coupled cluster energy is -20.656290717085593, with relative error 1.4376796420023652e-09
The Trotter UCCSD energy is -20.647635813749528, with relative error 0.00041899744437634556
The qsci gs energy is -20.656284795538582, with relative error 2.8810807384260784e-07
